<a href="https://colab.research.google.com/github/Ditsuhi/Graph_Neural_Networks_A3T_GCN/blob/main/Graph_Neural_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#install pytorch and all related libraries
#it will take some time

!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-${pt_version}.html
!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-${pt_version}.html
!pip install torch-cluster -f https://pytorch-geometric.com/whl/torch-${pt_version}.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-${pt_version}.html
!pip install torch-geometric
!pip install torch-geometric-temporal


Looking in links: https://pytorch-geometric.com/whl/torch-.html
  Created wheel for torch-scatter: filename=torch_scatter-2.0.9-cp37-cp37m-linux_x86_64.whl size=279639 sha256=3ef9926d98209564bf4c4b837a4811d133f2e5212401ba415d2b07b829fcb684
  Stored in directory: /root/.cache/pip/wheels/dd/57/a3/42ea193b77378ce634eb9454c9bc1e3163f3b482a35cdee4d1
Successfully built torch-scatter
Looking in links: https://pytorch-geometric.com/whl/torch-.html
     |████████████████████████████████| 48 kB 2.6 MB/s 
  Created wheel for torch-sparse: filename=torch_sparse-0.6.13-cp37-cp37m-linux_x86_64.whl size=501789 sha256=b747f09edfbcc03c225b057d08bc4e5ab3de45407f2171457c79eb71fc7537b9
  Stored in directory: /root/.cache/pip/wheels/e0/01/be/6b2966e0ff20bb023ae35e5d17903e6e5b4df46dd5892f6be6
Successfully built torch-sparse
Looking in links: https://pytorch-geometric.com/whl/torch-.html
     |████████████████████████████████| 43 kB 914 kB/s 
  Created wheel for torch-cluster: filename=torch_cluster-1.6.0-cp

In [ ]:
# import all required libraries

import pandas as pd
import numpy as np
import torch
from tensorflow import keras
from keras.layers import LSTM
from keras.layers import Dense
from keras.models import Sequential
from keras import layers
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from torch_geometric.utils import dense_to_sparse
from torch_geometric_temporal.signal import StaticGraphTemporalSignal
import torch.nn.functional as F
from torch_geometric_temporal.nn.recurrent import A3TGCN




In [4]:
#Creating adjacency matrix

data_adj= pd.read_csv('/content/distanceNodes.txt', encoding='utf8', delimiter='\t')

#create new column calling Distance_KM, which calculates distance between nodes in km
data_adj['Distance_KM']=data_adj['NEAR_DIST']/1000

#  Change the nodes id
replacement_mapping_dict = {
    0: 141, 1: 143, 2: 195, 3: 181, 4: 5, 5: 88, 6: 138, 7: 254, 8: 142, 9: 113, 10: 192, 11: 243, 12: 78, 13: 92, 
    14: 177, 15: 126, 16: 228, 17: 47, 18: 220, 19: 72, 20: 281, 21: 323, 22:217, 23:296
}
data_fin = data_adj[["IN_FID", "NEAR_FID"]].replace(replacement_mapping_dict)

# Replace columns with the new created columns
data_adj['IN_FID'] = data_fin["IN_FID"]
data_adj['NEAR_FID'] = data_fin["NEAR_FID"]

### create Adjacency matrix and replace column and index names according to grid cells id where air quality stations are located

am = pd.DataFrame(np.zeros(shape=(24, 24)))
am.rename(columns=replacement_mapping_dict, index =replacement_mapping_dict, inplace=True)
adj_mat = am.sort_index(axis=1)
adj_mat_complete = adj_mat.sort_index()


### Adjacency matrix

for i in data_adj.IN_FID.unique():
  for j in data_adj.NEAR_FID.unique():
    if i==j:
      adj_mat_complete.at[i,j]=0
    else:      
      adj_mat_complete.at[i,j]=1/data_adj.loc[(data_adj['IN_FID'] == i) & (data_adj['NEAR_FID'] == j)]['Distance_KM']
   
      
adj_mat_complete  

,5,47,72,78,88,92,113,126,138,141,...,192,195,217,220,228,243,254,281,296,323
5,0.000000,0.110356,0.236273,0.143935,0.181212,0.164228,0.112551,0.126298,0.117328,0.117353,...,0.091417,0.083470,0.066397,0.058654,0.075098,0.068696,0.054583,0.059056,0.058305,0.050814
47,0.110356,0.000000,0.109257,0.267139,0.095718,0.144390,0.208583,0.132625,0.079494,0.097987,...,0.087746,0.096880,0.102601,0.097302,0.082217,0.069394,0.084349,0.071076,0.064031,0.047170
72,0.236273,0.109257,0.000000,0.174832,0.645102,0.325035,0.148464,0.222798,0.221339,0.229932,...,0.147244,0.122364,0.082703,0.068796,0.107523,0.096458,0.064189,0.076388,0.076561,0.064459
78,0.143935,0.267139,0.174832,0.000000,0.145813,0.313177,0.442175,0.254499,0.113106,0.153721,...,0.126098,0.135770,0.116351,0.097973,0.109166,0.089090,0.086394,0.084737,0.077548,0.056153
88,0.181212,0.095718,0.645102,0.145813,0.000000,0.260419,0.133522,0.211733,0.330050,0.275207,...,0.166278,0.128027,0.081739,0.067254,0.115331,0.106098,0.063360,0.079434,0.081462,0.070607
92,0.164228,0.144390,0.325035,0.313177,0.260419,0.000000,0.271064,0.540075,0.176855,0.278847,...,0.180396,0.167196,0.109029,0.086970,0.133413,0.109384,0.079381,0.091725,0.087882,0.065522
113,0.112551,0.208583,0.148464,0.442175,0.133522,0.271064,0.000000,0.308841,0.114254,0.168040,...,0.149161,0.180343,0.157361,0.122457,0.135411,0.103990,0.105806,0.103478,0.091317,0.060882
126,0.126298,0.132625,0.222798,0.254499,0.211733,0.540075,0.308841,0.000000,0.180816,0.365729,...,0.247734,0.241840,0.131396,0.098479,0.175335,0.133346,0.089911,0.110489,0.104375,0.071900
138,0.117328,0.079494,0.221339,0.113106,0.330050,0.176855,0.114254,0.180816,0.000000,0.331141,...,0.223056,0.144352,0.083149,0.066961,0.139938,0.138718,0.064239,0.090053,0.097515,0.089265
141,0.117353,0.097987,0.229932,0.153721,0.275207,0.278847,0.168040,0.365729,0.331141,0.000000,...,0.409101,0.234668,0.109792,0.083599,0.198333,0.165686,0.078932,0.111642,0.114614,0.085327


In [5]:
# To open the file containing nodes features

data = pd.read_csv('/content/Madrid_Stations_2019.csv', header=None)
data_test = pd.read_csv('/content/Madrid_Stations_2020.csv', header=None)
fin_data=data.to_numpy().reshape(-1, 24, 20)
fin_data_test=data_test.to_numpy().reshape(-1, 24, 20)
fin_data_test.shape

(4367, 24, 20)

In [8]:
# select columns for further analysis, including 'NO2', 'intensidad', 'ocupacion', 'windSpeed', ' Pressure', ' SolarRad',' Temperature', ' Humidity', 'carga', 'vmed', 
# 'windDir_Categ_east', 'windDir_Categ_north', 'windDir_Categ_northeast', 'windDir_Categ_northwest', 'windDir_Categ_south',
#     'windDir_Categ_southeast', 'windDir_Categ_southwest', 'windDir_Categ_west'

fin_data_test = fin_data_test[:, :,np.r_[0:10, 12:20]]
fin_data = fin_data[:, :,np.r_[0:10, 12:20]]

In [9]:
# standardise train data

data = fin_data.transpose(
            (1, 2, 0)
        )
data = data.astype(np.float32)


# standardise (via Z-Score Method)
means = np.mean(data, axis=(0, 2))
data_norm= data- means.reshape(1, -1, 1)
stds = np.std(data_norm, axis=(0, 2))
data_norm= data_norm/ stds.reshape(1, -1, 1)
fin_data_norm = data_norm.transpose(2, 0, 1)
fin_data = data.transpose(2, 0, 1)


In [10]:
fin_data.shape

(4344, 24, 18)

In [11]:
# standardise test data

data = fin_data_test.transpose(
            (1, 2, 0)
        )
data = data.astype(np.float32)

data_norm= data- means.reshape(1, -1, 1)
data_norm= data_norm/ stds.reshape(1, -1, 1)
fin_data_test = data.transpose(2, 0, 1)
fin_data_test_norm = data_norm.transpose(2, 0, 1)

In [12]:
fin_data_test_norm.shape

(4367, 24, 18)

In [22]:
# split dataset to X and y (dependent and independent)

def split_sequence(sequence, seq_notNorm, time_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
   
		# find the end of this pattern
		end_ix = i +12
		time_steps_starter = 0  # it can be assigned as one of the following {0, 12, 24, 36}

		# check if we are beyond the sequence
		if end_ix+time_steps_starter+time_steps > len(sequence)-1:
			break
		# gather input and output parts of the pattern    
		seq_x, seq_y = sequence[i:end_ix], seq_notNorm[end_ix+time_steps_starter:end_ix+time_steps_starter+time_steps]
		X.append(seq_x)
		y.append(seq_y)
    
	return np.array(X), np.array(y)
 

# choose a number of time steps 
time_steps = 12
X_train, y_train = split_sequence(fin_data_norm, fin_data, time_steps)
X_test, y_test = split_sequence(fin_data_test_norm, fin_data_test, time_steps)

# to select only nitrogen dioxide as a target feature
y_train = y_train[:, :, :, 0]
y_test = y_test[:, :, :, 0]


In [23]:
y_test.shape

(4343, 12, 24)

In [24]:
X_train.shape

(4320, 12, 24, 18)

Create the models and run them (A3T-GCN and LSTM)

LSTM

In [28]:
# LSTM
# reshape the data for LSTM input

number_selected_columns =18
X_train_reshaped = X_train.reshape((X_train.shape[0], X_train.shape[1], 24*number_selected_columns))
X_test_reshaped = X_test.reshape((X_test.shape[0], X_test.shape[1], 24*number_selected_columns))


In [29]:
# define model

model = Sequential()
model.add(Dense(432,input_shape=(X_train.shape[1], 24*number_selected_columns)))
model.add(LSTM(512, return_sequences=True))
model.add(Dense(24))
model.compile(optimizer='adam', loss='mse')

In [ ]:
# run LSTM

lstm_model = model.fit(X_train_reshaped, y_train, epochs=50, verbose=2)


Epoch 1/50
134/134 - 36s - loss: 1097.7883 - 36s/epoch - 266ms/step
Epoch 2/50
134/134 - 37s - loss: 663.5463 - 37s/epoch - 274ms/step
Epoch 3/50
134/134 - 35s - loss: 522.7582 - 35s/epoch - 262ms/step
Epoch 4/50
134/134 - 37s - loss: 444.6629 - 37s/epoch - 276ms/step
Epoch 5/50
134/134 - 36s - loss: 378.6055 - 36s/epoch - 269ms/step
Epoch 6/50
134/134 - 36s - loss: 313.7090 - 36s/epoch - 270ms/step
Epoch 7/50
134/134 - 35s - loss: 258.5215 - 35s/epoch - 260ms/step
Epoch 8/50
134/134 - 36s - loss: 216.3493 - 36s/epoch - 269ms/step
Epoch 9/50
134/134 - 34s - loss: 193.1112 - 34s/epoch - 257ms/step
Epoch 10/50
134/134 - 35s - loss: 174.4701 - 35s/epoch - 261ms/step
Epoch 11/50
134/134 - 34s - loss: 164.2129 - 34s/epoch - 254ms/step
Epoch 12/50
134/134 - 36s - loss: 154.6314 - 36s/epoch - 267ms/step
Epoch 13/50
134/134 - 35s - loss: 147.3330 - 35s/epoch - 260ms/step
Epoch 14/50
134/134 - 37s - loss: 141.4970 - 37s/epoch - 273ms/step
Epoch 15/50
134/134 - 35s - loss: 135.4062 - 35s/epoch -

In [ ]:
y_test.shape

(4307, 12, 24)

In [ ]:
yhat = model.predict(X_test_reshaped, verbose=1)
yhat_reshaped = yhat.reshape(-1,24)
y_test_reshaped=  y_test.reshape(-1,24)
rmse = mean_squared_error(yhat_reshaped, y_test_reshaped, squared=False)
mae = mean_absolute_error(yhat_reshaped, y_test_reshaped)
print('Test Score: %.2f RMSE' % (rmse))
print('Test Score: %.2f MAE' % (mae))

135/135 [==============================] - 7s 52ms/step
Test Score: 19.72 RMSE
Test Score: 14.31 MAE


A3T-GCN

In [34]:
#to convert adjacency dataframe to numpy
adj = adj_mat_complete.to_numpy()

#transpose train data
data = fin_data.transpose(
            (1, 2, 0)
        )
data = data.astype(np.float32)


# standardise (via Z-Score Method)
means = np.mean(data, axis=(0, 2))
data_norm= data- means.reshape(1, -1, 1)
stds = np.std(data_norm, axis=(0, 2))
data_norm= data_norm/ stds.reshape(1, -1, 1)

#to convert adjacency matrix and data totorch
adj = torch.from_numpy(adj)
data_norm= torch.from_numpy(data_norm)
data= torch.from_numpy(data)

In [35]:
#transpose test data
data_test = fin_data_test.transpose(
            (1, 2, 0)
        )
data_test = data_test.astype(np.float32)


# Normalise as in DCRNN paper (via Z-Score Method)

data_test_norm= data_test - means.reshape(1, -1, 1)
data_test_norm = data_test_norm/ stds.reshape(1, -1, 1)


data_test= torch.from_numpy(data_test)
data_test_norm = torch.from_numpy(data_test_norm)

In [36]:
adj.shape

torch.Size([24, 24])

In [37]:
data_test.shape

torch.Size([24, 18, 4367])

In [ ]:
#from adjacency matrix extract the edge indices and edge weights

edge_indices, values = dense_to_sparse(adj)
edge_indices = edge_indices.numpy()
values = values.numpy()
edges = edge_indices
edge_weights = values

In [ ]:
edges.shape

(2, 552)

In [ ]:
edge_weights.shape

(552,)

In [ ]:
class MadridDatasetLoader(object):
    """The dataset is based on 24 stations (nodes) each having 20 features (nodal features) 
    and 276 edges connecting each pair of nodes, the edge weights are the distance between the edges.
    """

    def __init__(self, data, data_norm, edges, edge_weights):
        super(MadridDatasetLoader, self).__init__()
        self.data = data
        self.data_norm = data_norm
        self.edges = edges 
        self.edge_weights= edge_weights


    def _generate_task(self, num_timesteps_in: int = 6, num_timesteps_out: int = 6):
        """Uses the node features of the graph and generates a feature/target
        relationship of the shape
        (num_nodes, num_node_features, num_timesteps_in) -> (num_nodes, num_timesteps_out)
        

        Args:
            num_timesteps_in (int): number of timesteps the sequence model sees
            num_timesteps_out (int): number of timesteps the sequence model has to predict
        """
        time_steps_starter = 0  # it can be assigned as one of the following {0, 12, 24, 36}
        indices = [
            (i, i +time_steps_starter+ (num_timesteps_in + num_timesteps_out))
            for i in range(self.data.shape[2] - (time_steps_starter+num_timesteps_in + num_timesteps_out) + 1)
        ]
        print(indices)
        # Generate observations
        features, target = [], []
        for i, j in indices:
            features.append((self.data_norm[:, :, i : i + num_timesteps_in]).numpy())
            target.append((self.data[  :, 0, i + num_timesteps_in +time_steps_starter: j]).numpy())

        self.features = features
        self.targets = target

    def get_dataset(
        self, num_timesteps_in: int = 6, num_timesteps_out: int = 6
    ) -> StaticGraphTemporalSignal:
        """Returns data iterator for the dataset as an instance of the
        static graph temporal signal class.

        Return types:
            * **dataset** *(StaticGraphTemporalSignal)* - The forecasting dataset.
        """
        
        self._generate_task(num_timesteps_in, num_timesteps_out)
        dataset = StaticGraphTemporalSignal(
            self.edges, self.edge_weights, self.features, self.targets
        )

        return dataset

In [ ]:
loader = MadridDatasetLoader(data_test, data_test_norm, edges, edge_weights)
dataset_test = loader.get_dataset(num_timesteps_in=12, num_timesteps_out=12)
print("Dataset type:  ", dataset_test)
print("Number of samples / sequences: ",  len(set(dataset_test)))

[(0, 24), (1, 25), (2, 26), (3, 27), (4, 28), (5, 29), (6, 30), (7, 31), (8, 32), (9, 33), (10, 34), (11, 35), (12, 36), (13, 37), (14, 38), (15, 39), (16, 40), (17, 41), (18, 42), (19, 43), (20, 44), (21, 45), (22, 46), (23, 47), (24, 48), (25, 49), (26, 50), (27, 51), (28, 52), (29, 53), (30, 54), (31, 55), (32, 56), (33, 57), (34, 58), (35, 59), (36, 60), (37, 61), (38, 62), (39, 63), (40, 64), (41, 65), (42, 66), (43, 67), (44, 68), (45, 69), (46, 70), (47, 71), (48, 72), (49, 73), (50, 74), (51, 75), (52, 76), (53, 77), (54, 78), (55, 79), (56, 80), (57, 81), (58, 82), (59, 83), (60, 84), (61, 85), (62, 86), (63, 87), (64, 88), (65, 89), (66, 90), (67, 91), (68, 92), (69, 93), (70, 94), (71, 95), (72, 96), (73, 97), (74, 98), (75, 99), (76, 100), (77, 101), (78, 102), (79, 103), (80, 104), (81, 105), (82, 106), (83, 107), (84, 108), (85, 109), (86, 110), (87, 111), (88, 112), (89, 113), (90, 114), (91, 115), (92, 116), (93, 117), (94, 118), (95, 119), (96, 120), (97, 121), (98, 12

In [ ]:
loader = MadridDatasetLoader(data, data_norm, edges, edge_weights)
dataset = loader.get_dataset(num_timesteps_in=12, num_timesteps_out=12)
print("Dataset type:  ", dataset)
print("Number of samples / sequences: ",  len(set(dataset)))

[(0, 24), (1, 25), (2, 26), (3, 27), (4, 28), (5, 29), (6, 30), (7, 31), (8, 32), (9, 33), (10, 34), (11, 35), (12, 36), (13, 37), (14, 38), (15, 39), (16, 40), (17, 41), (18, 42), (19, 43), (20, 44), (21, 45), (22, 46), (23, 47), (24, 48), (25, 49), (26, 50), (27, 51), (28, 52), (29, 53), (30, 54), (31, 55), (32, 56), (33, 57), (34, 58), (35, 59), (36, 60), (37, 61), (38, 62), (39, 63), (40, 64), (41, 65), (42, 66), (43, 67), (44, 68), (45, 69), (46, 70), (47, 71), (48, 72), (49, 73), (50, 74), (51, 75), (52, 76), (53, 77), (54, 78), (55, 79), (56, 80), (57, 81), (58, 82), (59, 83), (60, 84), (61, 85), (62, 86), (63, 87), (64, 88), (65, 89), (66, 90), (67, 91), (68, 92), (69, 93), (70, 94), (71, 95), (72, 96), (73, 97), (74, 98), (75, 99), (76, 100), (77, 101), (78, 102), (79, 103), (80, 104), (81, 105), (82, 106), (83, 107), (84, 108), (85, 109), (86, 110), (87, 111), (88, 112), (89, 113), (90, 114), (91, 115), (92, 116), (93, 117), (94, 118), (95, 119), (96, 120), (97, 121), (98, 12

In [ ]:
# define A3T-GCN

class TemporalGNN(torch.nn.Module):
    def __init__(self, node_features, periods):
        super(TemporalGNN, self).__init__()
        # Attention Temporal Graph Convolutional Cell
        self.tgnn = A3TGCN(in_channels=node_features, 
                           out_channels=128, 
                           periods=periods)
        # Equals single-shot prediction
        self.linear = torch.nn.Linear(128, periods)

    def forward(self, x, edge_index,  edge_weight):
        """
        x = Node features for T time steps
        edge_index = Graph edge indices
        """
        h = self.tgnn(x, edge_index,  edge_weight)
        h = F.relu(h)
        h = self.linear(h)
        return h

TemporalGNN(node_features=18, periods=12)

TemporalGNN(
  (tgnn): A3TGCN(
    (_base_tgcn): TGCN(
      (conv_z): GCNConv(18, 128)
      (linear_z): Linear(in_features=256, out_features=128, bias=True)
      (conv_r): GCNConv(18, 128)
      (linear_r): Linear(in_features=256, out_features=128, bias=True)
      (conv_h): GCNConv(18, 128)
      (linear_h): Linear(in_features=256, out_features=128, bias=True)
    )
  )
  (linear): Linear(in_features=128, out_features=12, bias=True)
)

In [ ]:
# GPU support
device = torch.device('cpu') # cuda


# Create model and optimizers
model = TemporalGNN(node_features=18, periods=12).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
model.train()

print("Running training...")
for epoch in range(50): 
    loss = 0
    step = 0
    for snapshot in dataset:
        snapshot = snapshot.to(device)
        # Get model predictions
        y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
        # Mean Squared Error
        loss = loss + torch.mean((y_hat-snapshot.y)**2) 
        step += 1
        

    loss = loss / (step + 1)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    print("Epoch {} train MSE: {:.4f}".format(epoch, loss.item()))

Running training...
Epoch 0 train MSE: 2291.3374
Epoch 1 train MSE: 2283.6689
Epoch 2 train MSE: 2269.8486
Epoch 3 train MSE: 2247.3550
Epoch 4 train MSE: 2215.8552
Epoch 5 train MSE: 2176.4263
Epoch 6 train MSE: 2130.6013
Epoch 7 train MSE: 2080.2073
Epoch 8 train MSE: 2027.3589
Epoch 9 train MSE: 1973.9463
Epoch 10 train MSE: 1920.9659
Epoch 11 train MSE: 1868.7136
Epoch 12 train MSE: 1817.0372
Epoch 13 train MSE: 1765.7513
Epoch 14 train MSE: 1714.9067
Epoch 15 train MSE: 1664.6515
Epoch 16 train MSE: 1614.8817
Epoch 17 train MSE: 1565.2299
Epoch 18 train MSE: 1515.3132
Epoch 19 train MSE: 1464.8912
Epoch 20 train MSE: 1413.9109
Epoch 21 train MSE: 1362.4880
Epoch 22 train MSE: 1310.8330
Epoch 23 train MSE: 1259.1504
Epoch 24 train MSE: 1207.7296
Epoch 25 train MSE: 1157.0394
Epoch 26 train MSE: 1107.8077
Epoch 27 train MSE: 1060.7775
Epoch 28 train MSE: 1016.4797
Epoch 29 train MSE: 975.1299
Epoch 30 train MSE: 936.7495
Epoch 31 train MSE: 901.6166
Epoch 32 train MSE: 870.0934
Epoc

In [ ]:
model.eval()
loss = 0
step = 0

# Store for analysis
predictions = []
labels = []

for snapshot in dataset_test:
    snapshot = snapshot.to(device)
    # Get predictions
    y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
    # Mean Absolute Error
    loss = loss + torch.mean(torch.abs(y_hat-snapshot.y))
    # Store for analysis below
    labels.append(snapshot.y)
    predictions.append(y_hat)
    step += 1
  

loss = loss / (step+1)
loss = loss.item()
print("Test MAE: {:.4f}".format(loss))

In [ ]:
model.eval()
loss = 0
step = 0

# Store for analysis
predictions = []
labels = []

for snapshot in dataset_test:
    snapshot = snapshot.to(device)
    # Get predictions
    y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr)

    # Root Mean Squared Error
    loss = loss + torch.sqrt(torch.mean((y_hat-snapshot.y)**2)) 
    # Store for analysis below
    labels.append(snapshot.y)
    predictions.append(y_hat)
    step += 1

loss = loss / (step+1)
loss = loss.item()
print("Test RMSE: {:.4f}".format(loss))